In [1]:
import numpy as np
import pandas as pd
from math import *
from statistics import *
import yfinance as yf
from fin_lib import *
from Backtesting import *
import warnings
warnings.filterwarnings("ignore")



In [2]:
df = pd.read_excel('google.xlsx')
df.index = pd.to_datetime(df['Date'])
df = df.drop(columns=['Date','Open','High','Low','Adj Close'])
df['SMA5'] = df['Close'].rolling(5).mean()
df['SMA10'] = df['Close'].rolling(10).mean()
df['SMA15'] = df['Close'].rolling(15).mean()
df['SMA20'] = df['Close'].rolling(20).mean()
df = df.dropna()

In [3]:
## we have alpha = number of past days to include, beta = profit margin, gamma = number of future days to hold.
params = {
    'alpha': list(range(1,11)),
    'beta': [0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07],
    'gamma': list(range(5,21,5))

}

print("Number of Combinations = ",len(params['alpha'])*len(params['beta'])*len(params['gamma']))
keys = list(params.keys())


Number of Combinations =  440


In [4]:
df['SMA5TARGET'] = [0]*len(df)
df['SMA10TARGET'] = [0]*len(df)
df['SMA15TARGET'] = [0]*len(df)
df['SMA20TARGET'] = [0]*len(df)
df['COMBINED_SMA_TARGET'] = [0]*len(df)


In [ ]:
for i in range(len(df)):
    if df['SMA5'][i] > df['SMA10'][i]:
        df['SMA5TARGET'][i] = 1
    
    if df['SMA10'][i] > df['SMA15'][i]:
        df['SMA10TARGET'][i] = 1
    if df['SMA15'][i] > df['SMA20'][i]:
        df['SMA15TARGET'][i] = 1


    if (df['SMA5TARGET'][i] == 1) and (df['SMA10TARGET'][i] == 1) and (df['SMA15TARGET'][i] == 1):
        df['COMBINED_SMA_TARGET'][i] = 1
        

df = df.drop(columns=['SMA5','SMA10','SMA15','SMA20',])

In [5]:
num_batches = 5
rows_per_batch = len(df) // num_batches
batches = [df.iloc[i*rows_per_batch:(i+1)*rows_per_batch] for i in range(num_batches)]
if len(df) % num_batches != 0:
    batches[-1] = pd.concat([batches[-1], df.iloc[num_batches*rows_per_batch:]])

for i, batch in enumerate(batches):
    print(f"Batch {i+1}: {len(batch)} rows")


Batch 1: 751 rows
Batch 2: 751 rows
Batch 3: 751 rows
Batch 4: 751 rows
Batch 5: 751 rows


In [7]:
results_dic = {}
for r in range(len(batches)):
    df = pd.DataFrame()
    df = batches[r]
    model_results = {}
    counter=0

    for j in range(len(params[keys[1]])):
        beta = params[keys[1]][j]
        df = target_creation(df,beta)
        for z in range(len(params[keys[0]])):
            alpha = params[keys[0]][z]
            X_train_scaled,X_test_scaled,y_train,y_test = preprocessing(df,alpha)
            pred,precision = Models.MLP(X_train_scaled,X_test_scaled,y_train,y_test)
            for k in range(len(params[keys[2]])):
                gamma = params[keys[2]][k]
                current_vals = [gamma,len(X_train_scaled)]
                transaction_dict,result = Strategies.gamma_based_strategy(df,len(X_train_scaled),pred,gamma)
                current_params = [alpha,beta,gamma]
                
                counter+=1
                model_results.update({counter:[current_params,[precision],[float(result)]]})
                results_dic.update({r:model_results})


    

In [8]:
scores = []
for i in range(1, len(model_results)):
    param_array = []

    for j in range(len(results_dic)):
        current = results_dic[j][i][2]
        param_array.append(current[0])
        
        if len(param_array) == 5:
            scores.append(mean(param_array))
            param_array = []



In [10]:
for i in range(len(scores)):
    if scores[i] == max(scores):
        print(i+1)
results_dic[4][105]

105


[[7, 0.03, 5], [0.1111111111111111], [39.74000549316423]]